# Obtener empresas por industria de cada país

In [ ]:
# Requerimientos
!pip install selenium pandas sqlalchemy

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import sqlalchemy

db = sqlalchemy.create_engine("sqlite:///data.sqlite")

In [28]:
def get_companies(country):
    df = pd.read_sql(f"select * from {country}_industries", db)
    countries = []
    for row in df.values:
        sector = row[1]
        industry = row[2]
        url = row[3]

        driver = webdriver.Edge()
        driver.get(url)
        links = driver.find_elements(By.XPATH, "//td/span/a")
        names = driver.find_elements(By.XPATH, "//td/span/sup")

        for index, a in enumerate(links):
            countries.append({
                "sector": sector,
                "industry": industry,
                "ticker": a.text,
                "name": names[index].text,
                "href": a.get_property("href")
            })
    pd.DataFrame(countries).to_sql(f"{country}_companies", db, if_exists="replace")

for country in ["chile", "colombia", "mexico", "peru"]:
    get_companies(country)

In [2]:
df = pd.read_sql("select * from peru_companies", db)
df.tail()

,index,sector,industry,ticker,name,href
2,2,Servicios públicos,Eléctricas,ENGIEC1,ENGIE ENERGIA DEL PERU SA,https://es.tradingview.com/symbols/BVL-ENGIEC1/
3,3,Servicios públicos,Eléctricas,HIDRA2C1,EMPRESA REGIONAL ELECTRONORTE MEDIO,https://es.tradingview.com/symbols/BVL-HIDRA2C1/
4,4,Servicios públicos,Eléctricas,LUSURC1,LUZ DEL SUR SA,https://es.tradingview.com/symbols/BVL-LUSURC1/
5,5,Servicios públicos,Eléctricas,PERENBC1,PERUANA DE ENERGIA SA,https://es.tradingview.com/symbols/BVL-PERENBC1/
6,6,Servicios públicos,Eléctricas,SEALDC1,SOCIEDAD ELECTRICA DEL SUR OESTE SA,https://es.tradingview.com/symbols/BVL-SEALDC1/
